In [1]:
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import re
import csv

In [ ]:
# Effacer un projet
# bd.projects.delete_project('DOC_casestudies', delete_dir=True)

In [ ]:
# Supprimer la base de données
#del bd.databases['test']

In [ ]:
#bi.remote.install_project('ecoinvent-3.8-biosphere', 'DOC_casestudies')

In [4]:
bd.projects.set_current(name='DOC_casestudies')

In [ ]:
list(bd.projects)

In [ ]:
# Importer ecoinvent
bi.import_ecoinvent_release(
    version='3.8',
    system_model='cutoff',
    username='a',
    password='A'
)

In [ ]:
for method in bd.methods:
    #if 'IMPACT World+ Canada' in method[0]:
        #if 'Marine eutrophication' in method[2]:
            print(method)

In [ ]:
bi.BW2Package.import_file('methods/bw/impact_world_plus_Canada_ei38.c11cd6ab.bw2package')

In [ ]:
# Exporter les données d'une méthode d'EICV vers un fichier csv
# Les catégories d'impact qui t'intéressent
# World+
locations = ['Nunavik', 'Canada', 'World']
for loc in locations:
    categories_to_export = [
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater ecotoxicity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Human toxicity cancer'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Human toxicity non-cancer'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Particulate matter formation'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Marine eutrophication'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater eutrophication'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Terrestrial acidification'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater acidification'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Water scarcity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Land occupation, biodiversity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Land transformation, biodiversity')
    ]

    # Nom du fichier CSV de sortie
    csv_filename = f"impact_world_{loc}_midpoint_factors.csv"

    # Ouvre un fichier CSV en mode écriture
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Method Name', 'Flow ID', 'Flow Name', 'Category', 'Subcategory', 'Unit', 'Impact Factor']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Écrire l'en-tête
        writer.writeheader()
        

        # Parcours des méthodes et extraction des données
        for method_key in categories_to_export:
            print(method_key)
            try:
                method = bd.Method(method_key)
                method_data = method.load()
            
                for flow, impact_factor in method_data:
                    try:
                        # Récupérer les informations détaillées sur le flux élémentaire
                        activity = bd.get_activity(flow)
                    
                        # Écriture des données dans le CSV
                        writer.writerow({
                            'Method Name': str(method_key),
                            'Flow ID': flow,
                            'Flow Name': activity['name'],
                            'Category': activity['categories'][0] if activity['categories'] else None,
                            'Subcategory': activity['categories'][1] if len(activity['categories']) > 1 else None,
                            'Unit': activity['unit'],
                            'Impact Factor': impact_factor
                        })
                    except KeyError:
                        print(f"Flux introuvable pour l'identifiant {flow}")
            except bd.errors.UnknownObject:
                print(f"Méthode introuvable : {method_key}")


In [ ]:
# Créer une méthode sur la base de IW+ pour chaque région à modéliser
# Définir la méthode source et la méthode cible
old_method_name = 'IMPACT World+ Midpoint 2.0'
new_method_name = 'IMPACT World+ World'

# Définir les catégories d'intérêt
categories_of_interest = [
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Freshwater ecotoxicity'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Human toxicity cancer'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Human toxicity non-cancer'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Particulate matter formation'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Marine eutrophication'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Freshwater eutrophication'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Terrestrial acidification'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Freshwater acidification'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Water scarcity'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Land occupation, biodiversity'),
    ('IMPACT World+ Midpoint 2.0', 'Midpoint', 'Land transformation, biodiversity')
]

# Boucle sur les catégories d'intérêt
for category in categories_of_interest:
    # Charger les données de la méthode existante
    old_method = bd.Method(category)
    method_data = old_method.load()

    # Créer une nouvelle méthode avec la même catégorie dans la méthode cible
    new_method_key = (new_method_name, category[1], category[2])
    new_method = bd.Method(new_method_key)
    new_method.register()

    # Écrire les mêmes données dans la nouvelle méthode
    new_method.write(method_data)

    print(f"Copié {category[2]} de {old_method_name} vers {new_method_name}")


In [ ]:
# Modifier les méthodes pour chaque région d'après un fichier CSV
# Définir la méthode cible
#new_method_name = 'IMPACT World+ Nunavik'
#new_method_name = 'IMPACT World+ Canada'
new_method_name = 'IMPACT World+ World'

# Charger le fichier CSV dans un DataFrame pandas
#csv_filename = "methods/bw/IW_NU_16.csv"
#csv_filename = "methods/bw/IW_CA_15.csv"
csv_filename = "methods/bw/IW_WO_16.csv"

df = pd.read_csv(csv_filename)

# Remplacer les catégories spécifiques avant traitement
df['Category'] = df['Category'].replace({
    ' Land occupation': 'Land occupation, biodiversity',
    ' Land transformation': 'Land transformation, biodiversity'
})

# Dictionnaire pour stocker le nombre de mises à jour par catégorie
update_count_by_category = {}

# Liste des catégories d'intérêt
categories_of_interest = [
    'Freshwater ecotoxicity',
    'Human toxicity cancer',
    'Human toxicity non-cancer',
    'Particulate matter formation',
    'Marine eutrophication',
    'Freshwater eutrophication',
    'Terrestrial acidification',
    'Freshwater acidification',
    'Water scarcity',
    'Land occupation, biodiversity',
    'Land transformation, biodiversity'
]

# Parcourir les catégories d'intérêt et filtrer le DataFrame pour chaque catégorie
for category in categories_of_interest:
    # Créer un DataFrame pour chaque catégorie d'impact
    category_df = df[df['Category'].str.contains(category)]

    if category_df.empty:
        print(f"Aucune donnée pour la catégorie {category}.")
        continue

    # Nettoyer la catégorie si nécessaire
    cleaned_category = category.strip()

    # Traiter les flux pour cette catégorie
    method_key = (new_method_name, 'Midpoint', cleaned_category)
    try:
        # Charger la méthode correspondante
        method = bd.Method(method_key)
        method_data = method.load()

        # Rechercher les mises à jour pour chaque flux dans cette catégorie
        updated_data = []
        for flow, impact_factor in method_data:
            # Vérifier si ce flow_id est dans le DataFrame de la catégorie
            flow_row = category_df[category_df['Flow ID'] == flow]

            if not flow_row.empty:
                # Mettre à jour l'impact factor avec la valeur du CSV
                new_impact_factor = float(flow_row['Impact Factor'].values[0])
                updated_data.append((flow, new_impact_factor))
                update_count_by_category[cleaned_category] = update_count_by_category.get(cleaned_category, 0) + 1
            else:
                # Conserver l'ancien impact factor si non trouvé dans le CSV
                updated_data.append((flow, impact_factor))

        # Réécrire les données mises à jour dans la méthode
        method.write(updated_data)

    except Exception as e:
        print(f"Erreur lors du traitement de la catégorie {cleaned_category}: {e}")

# Afficher le nombre de flux mis à jour par catégorie
print("\nMises à jour par catégorie :")
for category, count in update_count_by_category.items():
    print(f"{category}: {count} flux mis à jour")


In [ ]:
# Exporter dans un bw2package
# Définir la liste des méthodes à exporter (catégories IMPACT World+ Nunavik)

locations = ['Nunavik', 'Canada', 'World']
for loc in locations:
    categories_to_export = [
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater ecotoxicity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Human toxicity cancer'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Human toxicity non-cancer'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Particulate matter formation'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Marine eutrophication'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater eutrophication'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Terrestrial acidification'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Freshwater acidification'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Water scarcity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Land occupation, biodiversity'),
        (f'IMPACT World+ {loc}', 'Midpoint', 'Land transformation, biodiversity')
    ]

    # Charger les objets Method correspondant aux catégories
    methods_to_export = [bd.Method(method_key) for method_key in categories_to_export]
    print(methods_to_export)

    # Définir le nom du fichier pour le package
    bw2package_filename = f'impact_world_plus_{loc}_ei38'

    # Exporter les méthodes dans un fichier .bw2package
    bi.package.BW2Package.export_objs(methods_to_export, filename=bw2package_filename, folder='/home/thcalken/Documents/LCA_project/methods/bw')

    print(f"Les méthodes ont été exportées dans {bw2package_filename}.bw2package")



In [ ]:
# Créer une base de données :
# case_studies = bd.Database('case_study_data').register()

In [ ]:
list(bd.databases)

In [5]:
eidb = bd.Database("ecoinvent-3.8-cutoff")

In [6]:
case_studies = bd.Database("case_study_data")

In [ ]:
# Lister toutes les activités dans ma bdd
for activity in case_studies.search('*'):
    print(f"{activity['name']}, {activity.key[1]}")

In [28]:
# Supprimer toutes les activités de la base de données
for activity in case_studies:
    activity.delete()

In [7]:
import uuid

# Générateur d'ID unique
def generate_unique_id(prefix=""):
    return f"{prefix}{uuid.uuid4().hex[:8]}"

In [29]:
# Créer les activités des systèmes modélisés
scenarios = ['electricity, baseline', 'electricity, alternative', 'heat, baseline', 'heat, alternative']
phases = ['production', 'distribution', 'use', 'end of life']
locations = ['onsite', 'offsite']

# Parcourir chaque combinaison de scénario et phase pour créer des activités
for scenario in scenarios:
    for location in locations:
        # Créer une nouvelle activité dans la base de données
        activity_name = f"{location}_{scenario}"
        new_activity = case_studies.new_activity(
            code=generate_unique_id(),  # Générer un code unique
            name=activity_name,
            unit="item",  # Unité en termes d'items
            type="process",  # Type d'activité
        )
        
        # Sauvegarder l'activité
        new_activity.save()
        for phase in phases:
            # Nom de l'activité combinant le scénario et la phase
            activity_name = f"{location}_{scenario}, {phase} phase"
    
            # Créer une nouvelle activité dans la base de données
            new_activity = case_studies.new_activity(
                code=generate_unique_id(),  # Générer un code unique
                name=activity_name,
                unit="item",  # Unité en termes d'items
                type="process",  # Type d'activité
            )
    
            # Sauvegarder l'activité
            new_activity.save()

In [30]:
# Ajouter les exchanges aux systèmes correspondants depuis mon excel

# Charger les données depuis l'Excel
df = pd.read_excel('activities_names.xlsx')

# Parcourir chaque ligne du fichier Excel
for index, row in df.iterrows():
    # Récupérer le nom de l'activité fils dans la première colonne
    child_name = row.iloc[0]
    
    # Trouver l'activité fils correspondante dans la base de données
    child_activity_candidates = [act for act in eidb if child_name in act['name'] and (act['location'] == 'RoW' or act['location'] == 'GLO')]
    
    if not child_activity_candidates:
        print(f"Activity '{child_name}' not found in eidb database.")
        continue
    else:
        # Compter le nombre de mots dans le nom de l'activité recherchée
        child_name_word_count = len(child_name.split())

        # Trouver l'activité avec le nombre de mots le plus proche
        closest_activity = min(child_activity_candidates, key=lambda act: abs(len(act['name'].split()) - child_name_word_count))

        # Assigner cette activité
        child_activity = closest_activity

        #print(f"Selected activity: '{child_activity['name']}' with {(len(child_activity['name'].split()))} words.")

    
    # Parcourir les colonnes restantes pour trouver les activités parent
    for parent_name, amount in row.iloc[1:].items():
        # Si une quantité est indiquée
        if not pd.isna(amount):
            # Trouver l'activité parent correspondante dans la base de données
            
            parent_activity = [act for act in case_studies if parent_name in act['name']]
            
            if not parent_activity:
                print(f"Parent activity '{parent_name}' not found in case studies database.")
                continue
            else:
                for prefix_parent_act in parent_activity:
            # Ajouter un nouvel exchange dans l'activité parent
                    new_exc = prefix_parent_act.new_exchange(
                        input=child_activity.key,  # Ajoute l'activité fils
                        amount=amount,  # Quantité tirée de l'Excel
                        type='technosphere',#child_activity['type'],  # Ou 'biosphere', selon le type d'échange
                        unit=child_activity['unit']
                    )
                    new_exc.save()  # Sauvegarde l'exchange

                print(f"Added exchange from '{child_activity['name']}' du type {child_activity['type']} to '{parent_name}' with amount {amount}{child_activity['unit']}.")


Added exchange from 'chimney production' du type process to 'heat, baseline, production phase' with amount 4.1825000000000003e-07meter.
Added exchange from 'chimney production' du type process to 'heat, baseline, end of life phase' with amount 4.1825000000000003e-07meter.
Added exchange from 'diesel, burned in diesel-electric generating set, 10MW' du type process to 'electricity, baseline, use phase' with amount 3.9240000000000004megajoule.
Added exchange from 'distribution network construction, electricity, low voltage' du type process to 'electricity, baseline, production phase' with amount 5.517241379310345e-09kilometer.
Added exchange from 'distribution network construction, electricity, low voltage' du type process to 'electricity, baseline, distribution phase' with amount 5.517241379310345e-09kilometer.
Added exchange from 'distribution network construction, electricity, low voltage' du type process to 'electricity, baseline, end of life phase' with amount 5.517241379310345e-09ki

In [31]:
# Trier les activités à inclure dans chaque phase et ajouter les exchanges correspondants dans les systèmes modélisés

# Dictionnaire contenant les critères spécifiques pour chaque phase
phase_criteria = {
    'prod': {
        'include': [],  # Critères à inclure
        'exclude': ['diesel, burned in building machine', 'excavation', 'market for electricity', 'market group for electricity']  # Critères à exclure
    },
    'dist': {
        'include': ['diesel, burned in building machine', 'excavation', 'market for electricity', 'market group for electricity'],  # Aucun critère à inclure
        'exclude': []  # Critère à exclure
    },
    'use': {
        'include': [],  # Critère à inclure
        'exclude': ['market for photovoltaic facade installation, 3kWp, multi-Si, panel, mounted, at building', 
                    'market for diesel-electric generating set, 10MW', 'market for ammonia, anhydrous, liquid',
                    'market for heavy fuel oil','market for limestone, crushed, washed',
                    'market for oil power plant, 500MW','market for chimney',
                    'market for oil boiler, 100kW','market for oil storage, 3000l', 
                    'market for furnace, pellets, with silo, 300kW']  # Aucun critère à exclure
    },
    'eol': {
        'include': ['municipal solid waste','waste plastic, mixture', 'waste polystyrene', 'waste polyvinylchloride', 'waste polyethylene'],  # Aucun critère à inclure
        'exclude': []  # Aucun critère à exclure
    }
}

scenario_activities = [act for act in case_studies if 'phase' in act['name']]

# Étape 1: Itérer sur les activités scénario-localisation-phase
for activity in scenario_activities:
    # Déterminer la phase pour chaque activité
    if 'production phase' in activity['name']:
        phase = 'prod'
    elif 'distribution phase' in activity['name']:
        phase = 'dist'
    elif 'use phase' in activity['name']:
        phase = 'use'
    elif 'end of life phase' in activity['name']:
        phase = 'eol'
    else:
        continue

    # Étape 2: Itérer sur les exchanges de chaque activité
    for exchange in activity.exchanges():
        sub_activity = exchange.input
        
        # Créer un nouveau nom avec un suffixe dépendant de la phase
        new_activity_name = sub_activity['name'] + f"_{phase}"
        new_activity = case_studies.new_activity(
            code=generate_unique_id('ph'), 
            name=new_activity_name,
            unit=sub_activity['unit'],  # Unité en termes d'items
            type=sub_activity['type']  # Type d'activité
            )
        new_activity.save()

        # Étape 3: Ajouter des exchanges aux nouvelles activités selon les critères spécifiques à la phase
        for sub_exc in sub_activity.exchanges():
            # Vérifier les critères positifs (include) et négatifs (exclude)
            include_check = not phase_criteria[phase]['include'] or any(criterion in sub_exc.input['name'] for criterion in phase_criteria[phase]['include'])
            exclude_check = not phase_criteria[phase]['exclude'] or not any(criterion in sub_exc.input['name'] for criterion in phase_criteria[phase]['exclude'])

            # Condition pour n'ajouter les valeurs négatives que dans la phase eol et use
            if sub_exc['amount'] < 0 and (phase == 'prod' or phase == 'dist'):
                continue  # Ignorer les exchanges négatifs dans les autres phases
        
            if include_check and exclude_check:
                sub_exc_type = sub_exc['type']
                if sub_exc_type != 'biosphere' and sub_exc_type != 'technosphere':
                    sub_exc_type = 'technosphere'
                new_exchange = new_activity.new_exchange(
                    amount=sub_exc['amount'],
                    input=sub_exc.input,
                    type=sub_exc_type,#sub_exc['type'],
                    unit=sub_exc['unit']
                )
                new_exchange.save()

        # Étape 4: Remplacer la sous-activité par la nouvelle activité dans l'exchange parent
        exchange.input = new_activity
        exchange.save()


In [32]:
# Créer les activités multirégionales & de end of life

multi_act_names = {'diesel, burned in building machine',
                   'excavation, hydraulic digger',
                   'excavation, skid-steer loader',
                   'market for municipal solid waste',
                   'market for waste plastic, mixture',
                   'market for waste polyethylene',
                   'market for waste polystyrene',
                   'market for waste polyvinylchloride',
                   'transport, freight, lorry >32 metric ton, EURO3',
                   'maintenance, lorry 40 metric ton',
                   'road construction',
                   'road maintenance'
                  }

for multi_name in multi_act_names:
    # Trouver l'activité fils correspondante dans la base de données
    multi_activity_candidates = [act for act in eidb if multi_name in act['name'] and (act['location'] == 'RoW' or act['location'] == 'GLO')]
    
    if not multi_activity_candidates:
        print(f"Activity '{multi_name}' not found in the database.")
        continue
    else:
        # Compter le nombre de mots dans le nom de l'activité recherchée
        multi_name_word_count = len(multi_name.split())

        # Trouver l'activité avec le nombre de mots le plus proche
        closest_activity = min(multi_activity_candidates, key=lambda act: abs(len(act['name'].split()) - multi_name_word_count))

        # Assigner cette activité
        multi_activity = closest_activity

        #print(f"Selected activity: '{multi_activity['name']}' with {(len(multi_activity['name'].split()))} words.")

                # Créer un nouveau nom avec un suffixe dépendant de la phase
        new_activity_name = multi_activity['name'] + f"_cs"
        new_activity = case_studies.new_activity(
            code=generate_unique_id('ws'), 
            name=new_activity_name,
            unit=multi_activity['unit'],  # Unité en termes d'items
            type=multi_activity['type']  # Type d'activité
        )
        new_activity.save()
        nb_exc = 0
        # Étape 3: Ajouter des exchanges aux nouvelles activités selon les critères spécifiques à la phase
        for exc in multi_activity.exchanges():
            new_exchange = new_activity.new_exchange(
                amount=exc['amount'],
                input=exc.input,
                type=exc['type'],
                unit=exc['unit']
            )
            new_exchange.save()
            nb_exc += 1
        #print(f"Saved activity: '{multi_activity['name']}' with {nb_exc} exchanges.")

In [33]:
# Additionner les échanges avec le même nom dans le RoW ou le GLO

# Parcourir chaque activité dans la base de données
for activity in case_studies:
    # Dictionnaire pour stocker les échanges par nom
    exchanges_by_name = {}

    # Itérer à travers les échanges de l'activité
    for exc in activity.exchanges():
        name = exc.input.get('name')  # Utiliser .get() pour éviter KeyError
        
        if name is None:
            continue  # Passer à l'échange suivant si le nom est absent
        
        # Ajouter l'échange à la liste correspondante à ce nom
        if name not in exchanges_by_name:
            exchanges_by_name[name] = []
        exchanges_by_name[name].append(exc)

    # Parcourir chaque groupe d'échanges par nom
    for name, exchanges in exchanges_by_name.items():
        if len(exchanges) > 1:  # Si plusieurs échanges avec le même nom
            row_exchange = None
            total_amount = 0
            world_loc_exists = False

            for exc in exchanges:
                total_amount += exc.get('amount', 0)  # Utiliser .get() pour éviter KeyError

                if exc.input.get('location') == 'RoW' or exc.input.get('location') == 'GLO':
                    row_exchange = exc  # Garder l'échange avec location 'RoW'
                    world_loc_exists = True
                    
            if not world_loc_exists:
                    row_exchange = exchanges[0]
                
            if row_exchange:
                row_exchange['amount'] = total_amount  # Mettre à jour le montant
                row_exchange.save()
                #print(f"Activity '{activity['name']}' - Summing exchanges '{name}' to location '{row_exchange.input.get('location')}'. New amount: {total_amount}")

            # Supprimer les autres échanges avec ce nom
            for exc in exchanges:
                if exc != row_exchange:
                    exc.delete()

    activity.save()

In [34]:
# Remplacer les activités d'eidb par mes activités multirégionales, et gestion cas spécifiques

multi_acts = [act for act in case_studies if '_cs' in act['name'] and not 'waste' in act['name']]
snp_acts = [act for act in case_studies if not '_prod' in act['name'] and not 'phase' in act['name']]

fallback_act = [act for act in case_studies if 'municipal solid waste' in act['name']][0]
lorry_act = [act for act in case_studies if 'lorry' in act['name'] and not 'maintenance' in act['name']][0]
road_act = [act for act in case_studies if 'road' in act['name'] and 'construction' in act['name']][0]

keyword_to_process = {
    'municipal solid waste': 'market for municipal solid waste_cs',
    'plastic, mixture': 'market for waste plastic, mixture_cs',
    'polystyrene': 'market for waste polystyrene_cs',
    'polyvinylchloride': 'market for waste polyvinylchloride_cs',
    'polyethylene': 'market for waste polyethylene_cs'
}

# Étape 1: Parcourir les activités avec le suffixe '_cs'
for multi_act in multi_acts:
    # Obtenir la liste des mots sans le suffixe '_cs'
    #base_words = multi_act['name'].replace('_cs', '').strip().split()
    # Obtenir la liste des mots sans le suffixe '_cs' et retirer les ','
    base_words = [word.replace(',', '').strip() for word in multi_act['name'].replace('_cs', '').split()]

    #print(f"Multi : {base_words}")
    
    # Étape 2: Parcourir les autres activités et vérifier les exchanges
    for snp_act in snp_acts:
        for exchange in snp_act.exchanges():
            # Découper le nom de l'exchange en mots
            exchange_words = exchange.input['name'].split()
            old_name = exchange.input['name']
            
            # Vérifier si tous les mots de l'activité sans suffixe sont présents dans l'exchange
            if all(word in exchange.input['name'] for word in base_words):
                # Étape 3: Remplacer l'exchange par l'activité avec le suffixe '_cs'
                exchange.input = multi_act
                exchange.save()
                #print(f"Replaced {old_name} with {multi_act['name']} in '{snp_act['name']}'")
            elif 'transport, freight, lorry' in exchange.input['name']:
                exchange.input = lorry_act
                exchange.save()
                #print(f"Replaced {old_name} with {lorry_act['name']} in '{snp_act['name']}'")
            elif 'road' in exchange.input['name'] and not 'maintenance' in exchange.input['name']:
                exchange.input = road_act
                exchange.save()
                #print(f"Replaced {old_name} with {road_act['name']} in '{snp_act['name']}'")
            # Vérifier seulement les échanges avec un montant négatif
            elif exchange.get('amount',0) < 0 and not 'treatment of' in exchange.input['name']:
                found_keyword = False  # Pour savoir si un mot-clé a été trouvé
                for keyword, new_process in keyword_to_process.items():
                    if keyword in exchange.input['name']:
                        # Remplacer l'échange par le nouveau processus
                        new_input_activity = [act for act in case_studies if new_process in act['name']][0]  # Trouver le nouveau processus
                        exchange['input'] = new_input_activity.key  # Mettre à jour la référence de l'échange
                        exchange.save()  # Enregistrer l'échange mis à jour
                        #print(f"Replaced {old_name} with {new_input_activity['name']} in '{snp_act['name']}'")
                        found_keyword = True  # Un mot-clé a été trouvé et traité
                        break

                # Si aucun mot-clé n'a été trouvé, utiliser fallback_act
                if not found_keyword and 'municipal solid waste' not in exchange.input['name']:
                    exchange['input'] = fallback_act.key
                    exchange.save()  # Enregistrer l'échange mis à jour
                    #print(f"Replaced {old_name} with {fallback_act['name']} in '{snp_act['name']}'")


In [35]:
# Garder seulement les open dump, wet infiltration class

waste_activities = [act for act in case_studies if '_cs' in act['name'] and 'waste' in act['name']]

for activity in waste_activities:
    # Skip the activity if it's in the excluded activities list
    od500mm_exchange = None
    total_amount = 0
    for exc in activity.exchanges():
        if exc['amount']<0:
            total_amount += exc.get('amount', 0)  # Utiliser .get() pour éviter KeyErro
            if 'open dump' in exc.input['name'] and '500mm' in exc.input['name']:
                od500mm_exchange = exc
            if od500mm_exchange:
                od500mm_exchange['amount'] = total_amount  # Mettre à jour le montant
                od500mm_exchange.save()
                #print(f"Activity '{activity['name']}' - Summing exchanges to open dump 500mm. New amount: {total_amount}")
            if exc != od500mm_exchange:
                exc.delete()
    activity.save()

In [36]:
# Ajouter les distances en bateau aux consommables

snp_acts = [act for act in case_studies if not '_prod' in act['name'] and not 'phase' in act['name']]

# Echanges à ajouter
petroleum_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, tanker for petroleum' in activity['name']
][0]
container_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, container ship' in activity['name']
    and not 'with' in activity['name']
][0]

keywords_all = {'market', 'for'}
keywords_oil = {'diesel', 'light fuel oil'}
keywords_other = {'wood pellet', 'lubricating oil'}

for act in snp_acts:
    for exc in act.exchanges():
        # Vérifier si tous les mots-clés de keywords_all sont dans le nom de l'exchange
        if all(keyword in exc.input['name'] for keyword in keywords_all):
            if exc['unit'] in ['kilogram', 'cubic meter']:
                # Vérifier si l'un des mots-clés de keywords_oil est présent dans le nom
                if any(keyword in exc.input['name'] for keyword in keywords_oil):
                    act.new_exchange(
                        input=petroleum_ship,
                        amount=(exc['amount'] * 3),
                        type=exc['type'],
                        unit=petroleum_ship['unit']
                    ).save()
                    #print(f"Added {petroleum_ship['name']}, {exc['amount'] * 3} {exc.unit} to {exc.input['name']} in {act['name']}")
                # Vérifier si l'un des mots-clés de keywords_other est présent dans le nom
                elif any(keyword in exc.input['name'] for keyword in keywords_other):
                    act.new_exchange(
                        input=container_ship,  # Correction ici, on ajoute container_ship
                        amount=(exc['amount'] * 3),
                        type=exc['type'],
                        unit=container_ship['unit']
                    ).save()
                    #print(f"Added {container_ship['name']}, {exc['amount'] * 3} {exc.unit}  to {exc.input['name']} in {act['name']}")
        

In [37]:
# Créer les activités onsite_ et offsite_

snp_acts = [act for act in case_studies if not '_prod' in act['name'] and not 'phase' in act['name']] #
multi_acts = [act for act in case_studies if '_cs' in act['name']]

# Créer deux activités fils pour chaque activité parent dans snp_acts
for parent_act in snp_acts:
    if not 'onsite_' in parent_act['name'] and not 'offsite_' in parent_act['name']:
        # Créer les activités fils onsite_ et offsite_
        onsite_act = case_studies.new_activity(
            code=generate_unique_id('onsite'),
            name=f"onsite_{parent_act['name']}",
            unit='process',#parent_act['unit'],  # Unité 
            type=parent_act['type']  # Type d'activité
        )
        offsite_act = case_studies.new_activity(
            code=generate_unique_id('offsite'),
            name=f"offsite_{parent_act['name']}",
            unit='process',#parent_act['unit'],  # Unité
            type=parent_act['type']  # Type d'activité
        )
        
        onsite_act.save()
        offsite_act.save()
        #print(f"Création d'activités pour {parent_act['name']}")
        n_bio = 0
        n_multi = 0
        n_off = 0
    
        # Copier les échanges de l'activité parent vers les activités fils
        for exc in parent_act.exchanges():
            # Si l'exchange est vers la biosphère (par exemple, type 'biosphere')
    
            if exc['type'] == 'biosphere':
                onsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='biosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                n_bio += 1
            
            # Si l'exchange est vers une activité de multi_acts
            elif exc.input in multi_acts:
                # Ajouter l'exchange dans les deux activités fils
                onsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='technosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                offsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='technosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                n_multi += 1

            # Electricité
            elif exc['unit'] == 'kilowatt hour':
                # Ajouter l'exchange dans les deux activités fils
                onsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='technosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                offsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='technosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                n_multi += 1
                
            # Tous les autres échanges
            else:
                offsite_act.new_exchange(
                    input=exc.input,
                    amount=exc['amount'],
                    type='technosphere',#exc['type'],
                    unit=exc['unit']
                ).save()
                n_off += 1

    #print(f"Copied {n_bio} biosphere, {n_multi} multiregional and {n_off} offsite exchanges to {parent_act['name']}")



In [38]:
# Mettre les exchanges onsite_ et offsite_ dans les activités sauf production phase

snp_acts = [act for act in case_studies if not '_prod' in act['name']] #and not 'phase' 

for parent_act in snp_acts:
    # Déterminer si l'activité parent est onsite_ ou offsite_
    if parent_act['name'].startswith('onsite_'):
        prefix = 'onsite_'
    elif parent_act['name'].startswith('offsite_'):
        prefix = 'offsite_'
    else:
        continue  # Passer à l'itération suivante si l'activité n'a pas le bon préfixe

    # Parcourir les exchanges de l'activité parent
    for exc in parent_act.exchanges():
        # Vérifier si l'exchange pointe vers une autre activité fils de snp_acts
        target_acts = [act for act in snp_acts if act['name'] == exc.input['name']]

        if target_acts:
            # Si l'activité parent est onsite_, faire pointer vers l'activité fils avec le préfixe onsite_
            if prefix == 'onsite_':
                target_onsite = [act for act in snp_acts if f"onsite_{target_acts[0]['name']}" == act['name']]
                if len(target_onsite)>0:
                    onsite_act = target_onsite[0]
                    #print(onsite_act)
                    parent_act.new_exchange(
                        input=target_onsite[0],
                        amount=exc['amount'],
                        type='technosphere',#exc['type'],
                        unit=exc['unit']
                    ).save()
                    exc.delete()
                    #print(f"Exchange mis à jour dans {parent_act['name']} pour pointer vers {target_onsite[0]['name']}")
            
            # Si l'activité parent est offsite_, faire pointer vers l'activité fils avec le préfixe offsite_
            elif prefix == 'offsite_':
                target_offsite = [act for act in snp_acts if f"offsite_{target_acts[0]['name']}" == act['name']]
                if len(target_offsite)>0:
                    offsite_act = target_offsite[0]
                    #print(offsite_act)
                    parent_act.new_exchange(
                        input=target_offsite[0],
                        amount=exc['amount'],
                        type='technosphere',#exc['type'],
                        unit=exc['unit']
                    ).save()
                    exc.delete()
                    #print(f"Exchange mis à jour dans {parent_act['name']} pour pointer vers {target_offsite[0]['name']}")


In [ ]:
# Création du process de distribution

scenarios_values = {
    'electricity, baseline': [989472, 58550.613, 128145],
    'electricity, alternative': [2100, 16590],
    'heat, baseline': [2280],
    'heat, alternative': [23718]
}

container_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, container ship' in activity['name']
    and not 'with' in activity['name']
][0]

# Création des activités sans échanges
for scenario, values in scenarios_values.items():
    for idx, value in enumerate(values):
        # Créer un nom unique pour l'activité
        activity_name = f"offsite_distribution {scenario}_{idx}"
        
        # Créer une nouvelle activité
        new_activity = case_studies.new_activity(
            code=generate_unique_id('act'),
            name=activity_name,
            unit='item',  # Spécifie l'unité
            type='process'  # Spécifie le type
        )
        
        # Sauvegarder l'activité sans échanges
        new_activity.save()

        # Stocker la nouvelle activité pour l'utiliser dans le second bloc
        print(f"Created activity: {new_activity['name']}")

# Ajout des échanges aux activités créées
for scenario, values in scenarios_values.items():
    for idx, value in enumerate(values):
        # Récupérer l'activité par son nom
        activity_name = f"offsite_distribution {scenario}_{idx}"
        new_activity = [
            act for act in case_studies if act['name'] == activity_name
        ][0]

        # Ajouter un échange à l'activité
        new_activity.new_exchange(
            amount=value,  # Valeur associée à l'activité
            input=container_ship,  # Spécifie l'input
            type='technosphere',  # Spécifie le type d'échange
            unit='ton kilometer'  # Spécifie l'unité
        ).save()

        # Sauvegarder l'activité après avoir ajouté les échanges
        new_activity.save()

        print(f"Added exchange to activity: {new_activity['name']} with value: {value}")

In [ ]:
# Ajouter les activités de distribution aux systèmes modélisés

scenarios = ['electricity, baseline', 'electricity, alternative', 'heat, baseline', 'heat, alternative']

for scenario in scenarios:
    dist_phase = [
        activity for activity in case_studies
        if scenario in activity['name']
        and 'distribution phase' in activity['name']
        and 'offsite' in activity['name']
    ][0]
    dist_acts = [
        activity for activity in case_studies
        if scenario in activity['name']
        and 'offsite_distribution' in activity['name']
    ]
    for act in dist_acts:
        dist_phase.new_exchange(
            amount=1,  # Valeur associée à l'activité
            input=act,  # Spécifie l'input, ici l'activité container_ship par exemple
            type='technosphere',  # Spécifie le type d'échange, ici technosphère par exemple
            unit=act['unit']#'item'  # Unité de l'échange, ici par exemple tonne.kilometer
        ).save()

In [ ]:
# Remplacer les process d'électricité
onsite_elec_act = [activity for activity in case_studies
    if 'electricity, baseline' in activity['name']   
    and not 'phase' in activity['name']
    and not 'distribution' in activity['name']  
    and 'onsite' in activity['name']
][0]

offsite_elec_act = [activity for activity in case_studies
    if 'electricity, baseline' in activity['name']   
    and not 'phase' in activity['name']
    and not 'distribution' in activity['name']  
    and 'offsite' in activity['name']
][0]

for activity in case_studies:
    for exchange in activity.exchanges():
        if exchange['type'] != 'biosphere' and exchange['type'] != 'production':
            if 'market' in exchange.input['name'] and 'electricity' in exchange.input['name']:
                if exchange['unit'] == 'kilowatt hour':
                    if not '_prod' in activity['name']:
                        old_exc_name = exchange.input['name']
                        if 'onsite' in activity['name']:
                            exchange.input = onsite_elec_act
                            exchange.save()
                            #print(f'Changed electricity in {activity['name']}')
                        elif 'offsite' in activity['name']:
                            exchange.input = offsite_elec_act
                            exchange.save()
                            #print(f'Changed electricity in {activity['name']}')


In [ ]:
# Ajouter elec, baseline use & eol à elec alternative

alt_elec_phases = [act for act in case_studies if 'electricity, alternative' in act['name'] and ('use' in act['name'] or 'end of life' in act['name'])]
for activity in alt_elec_phases:
    phase = activity['name'].split(',')[2]
    location = activity['name'].split('_')[0]
    matching_act = [act for act in case_studies if 'electricity, baseline' in act['name'] and phase in act['name'] and location in act['name']][0]
    activity.new_exchange(
        amount=0.382,  # Valeur associée à l'activité
        input=matching_act,  # Spécifie l'input, ici l'activité container_ship par exemple
        type='technosphere',  # Spécifie le type d'échange, ici technosphère par exemple
        unit='item'  # Unité de l'échange, ici par exemple tonne.kilometer
    ).save()
    print(f'Added {matching_act['name']} to {activity['name']}')


In [ ]:
# Populer les activités de plus haut niveau
top_process_names = ['electricity, baseline',
                     'electricity, alternative',
                     'heat, baseline',
                     'heat, alternative']
phases = ['production', 'distribution', 'use', 'end of life']

for name in top_process_names:
    top_level_act = [act for act in case_studies if name in act['name'] and not any(phase in act['name'] for phase in phases)]
    phase_level_act = [act for act in case_studies if name in act['name'] and 'phase' in act['name']]
    for act_t in top_level_act:
        location_t = act_t['name'].split('_')[0]
        for act_p in phase_level_act:
            location_p = act_p['name'].split('_')[0]
            if location_p == location_t:
                act_t.new_exchange(
                    amount=1,
                    input=act_p,
                    type='technosphere',
                    unit='item'
                ).save()
                # print(f'Added {act_p['name']} to {act_t['name']}')
    

In [55]:
# Créer les échanges de type production
prod_counter = 0
bad_prod_counter = 0
new_prod_counter = 0

for activity in case_studies:
    exc_list = activity.exchanges()
    exc_names = []
    prod_exists = False
    bad_prod_deleted = False
    for exc in exc_list:
        if exc['type'] == "production":
            if exc.input['name']!= activity['name']:
                exc.delete()
                bad_prod_deleted = True
                continue
            else:
                prod_exists = True
        exc_names.append((exc.input['name'], exc['type']))


    
#    for name, exc_type in exc_names:    
 #       if exc_type == "production":
  #          if name == activity['name']:
   #             prod_exists = True
    #            break
     #       else:
      #          bad_prod_exists = True
    
    if prod_exists:
        prod_counter += 1
#        print(f"Production exchange exists in {activity['name']}")
    elif bad_prod_deleted:
        bad_prod_counter += 1
#        print(f"Bad production exchange exists in {activity['name']}")
    else:
        # Ajouter un nouvel échange de production
        activity.new_exchange(
            amount=1,
            input=activity,
            type='production',
            unit=activity['unit']
        ).save()
        new_prod_counter += 1
#        print(f"Added production exchange to {activity['name']}")
print(prod_counter, bad_prod_counter, new_prod_counter)

222 0 0


In [ ]:
# Vérification top level activities
top_level_acts = [act for act in case_studies if any(name in act['name'] for name in top_process_names) and not any(phase in act['name'] for phase in phases)]
for act in top_level_acts:
    print(act['name'])
    for exc in act.exchanges():
        print(exc.input['name'])

In [56]:
# Sélectionner une activité dans une bdd
activity_1 = [
    activity for activity in case_studies
    if 'offsite' in activity['name']
    if 'phase' in activity['name']   
#    if 'offsite_distribution electricity, baseline_0' in activity['name']
#    and 'onsite' in activity['name']
#    and 'distribution' in activity['name']  

#    and '300' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
print(activity_1['name'], activity_1['type'], activity_1['unit'])
for exc in activity_1.exchanges():
 #   sub_act = [act for act in eidb if exc.input['name'] in act['name']][0]
  #  for exchange in sub_act.exchanges():
        print(exc.input['name'], exc['type'], exc['amount'], exc['unit'])

offsite_heat, baseline, use phase process item
offsite_heat production, light fuel oil, at boiler 100kW, non-modulating_use technosphere 1.0 megajoule
offsite_heat, baseline, use phase production 1 item


In [48]:
# Sélectionner une activité dans une bdd
activity_2 = [
    activity for activity in eidb
    if 'transport, freight, sea, container ship' in activity['name']
    and not 'reefer' in activity['name']   
    and not 'market' in activity['name']   
    and activity['location'] == 'GLO'
][0]
#for exc in activity_2.exchanges():
 #   print(exc.

In [41]:
Nu_methods = [method for method in bd.methods if 'IMPACT World+ Nunavik' in method[0]]
Ca_methods = [method for method in bd.methods if 'IMPACT World+ Canada' in method[0]]
Wo_methods = [method for method in bd.methods if 'IMPACT World+ World' in method[0]]

In [61]:
lca = bc.LCA({activity_1:1}, Nu_methods[0])
lca.lci()
lca.lcia()

print(lca.score)

nan


In [ ]:
# ACV
top_process_names = ['electricity, baseline',
                     'electricity, alternative',
                     'heat, baseline',
                     'heat, alternative']

phases = ['production', 'distribution', 'use', 'end of life']

top_level_acts = [act for act in case_studies if any(name in act['name'] for name in top_process_names) and not any(phase in act['name'] for phase in phases)]

# Créer un DataFrame vide pour stocker les résultats de l'ACV
results_df = pd.DataFrame(columns=['Process Name', 'Method', 'Impact Category', 'Score'])

Nu_methods = [method for method in bd.methods if 'IMPACT World+ Nunavik' in method[0]]
Ca_methods = [method for method in bd.methods if 'IMPACT World+ Canada' in method[0]]
Wo_methods = [method for method in bd.methods if 'IMPACT World+ World' in method[0]]
IW_mp_methods = [method for method in bd.methods if 'IMPACT World+ Midpoint 2.0' in method[0]]

def run_lca(act, location):
    if location == 'onsite':
        for method in Nu_methods:
                print(dir(method))
                lca = bc.LCA({act:1}, method)
                lca.lci()
                lca.lcia()
                results_df.loc[len(results_df)] = [act['name'], method[0], method[2], lca.score]
        for method in Ca_methods:
                lca = bc.LCA({act:1}, method)
                lca.lci()
                lca.lcia()
                results_df.loc[len(results_df)] = [act['name'], method[0], method[2], lca.score]

        for method in Wo_methods:
                lca = bc.LCA({act:1}, method)
                lca.lci()
                lca.lcia()
                results_df.loc[len(results_df)] = [act['name'], method[0], method[2], lca.score]

    elif location == 'offsite':
            for method in Wo_methods:
                lca = bc.LCA({act:1}, method)
                lca.lci()
                lca.lcia()
                results_df.loc[len(results_df)] = [act['name'], method[0], method[2], lca.score]

for act in top_level_acts:
    location = act['name'].split('_')[0]
    run_lca(act, location)
    for exc in act.exchanges():
        location = exc.input['name'].split('_')[0]
        run_lca(exc.input, location)

# Exporter les résultats dans un fichier CSV
results_df.to_csv('acv_results.csv', index=False, encoding='utf-8')
    


In [ ]:
# Afficher les scénarios triés et les exchanges associés

phase_activities = [act for act in case_studies if 'phase' in act['name'] and 'dist' in act['name']]
phase_order = ['production phase', 'distribution phase', 'use phase', 'end of life phase']

def get_scenario_phase_location(activity):

    parts_undr = activity['name'].split('_')
    location = parts_undr[0]
    parts_coma = parts_undr[1].split(', ')
    scenario = parts_coma[0] + parts_coma[1]
    phase = parts_coma[2]
    return scenario, location, phase

# Trier la liste des activités
sorted_activities = sorted(
    phase_activities,
    key=lambda act: (
        get_scenario_phase_location(act)[0],  # Trier par scénario
        get_scenario_phase_location(act)[1],  # Trier par localisation
        phase_order.index(get_scenario_phase_location(act)[2])  # Trier par phase selon l'ordre défini
    )
)

# Afficher les activités triées
for activity in sorted_activities:
    print(f"Activity: {activity['name']}, {activity['unit']}")
    for exc in activity.exchanges():
        print(f"  - {exc.input['name']}, {exc['amount']}{exc['unit']}")
        sub_exc_act = [act for act in case_studies if exc.input['name'] in act['name']][0]# and exc.input['location'] == 'RoW'][0]
        for sub_exc in sub_exc_act.exchanges():
            print(f"    - {sub_exc.input['name']}, {sub_exc['amount']}{sub_exc['unit']}")
    print(' ')

In [ ]:
# Générer l'arborescence de chaque scénario dans un csv
# Définir les phases dans l'ordre
phases = ['production phase', 'distribution phase', 'use phase', 'end of life phase']
scenarios = ['electricity, baseline', 'electricity, alternative', 'heat, baseline', 'heat, alternative']
locations = ['offsite', 'onsite']

def process_exchanges_dynamic(exchanges, data, level=1, max_level=15, explored=None):
    """
    Traite les échanges et ajoute dynamiquement des colonnes au DataFrame pour chaque niveau.
    
    :param exchanges: Liste d'échanges à traiter.
    :param data: DataFrame dans lequel ajouter les lignes.
    :param level: Niveau d'échange en cours de traitement (commence à 1).
    :param max_level: Niveau maximum à explorer pour éviter la récursion excessive.
    :param explored: Ensemble pour suivre les échanges déjà traités.
    :return: DataFrame mis à jour avec les nouveaux échanges et colonnes.
    """
    if explored is None:
        explored = set()  # Initialiser l'ensemble pour les échanges explorés

    # Vérifier si le niveau maximum est atteint
    if level > max_level:
        return data
    
    # Définir les colonnes pour ce niveau
    col_name = f'Exchange {level} (Name)'
    col_amount = f'Exchange {level} (Amount)'
    col_unit = f'Exchange {level} (Unit)'

    # Si ces colonnes n'existent pas dans le DataFrame, les ajouter
    if col_name not in data.columns:
        data[col_name] = ''
        data[col_amount] = ''
        data[col_unit] = ''
    for exc in [e for e in exchanges if e['type'] != 'production' and e['type'] != 'biosphere'] :
        # Vérifier si l'échange provient de la base de données case_studies
        

        

        # Créer une nouvelle ligne vide
        row = [''] * len(data.columns)
        row[data.columns.get_loc(col_name)] = exc.input['name']
        row[data.columns.get_loc(col_amount)] = exc['amount']
        row[data.columns.get_loc(col_unit)] = exc['unit']
        
        # Ajouter la nouvelle ligne au DataFrame
        data = pd.concat([data, pd.DataFrame([row], columns=data.columns)], ignore_index=True)
        if exc.input['database'] != 'case_study_data':
            continue  # Passer cet échange si ce n'est pas le bon type
        # Vérifier si cet échange a déjà été exploré
        exc_id = (exc.input['name'], exc['amount'], exc['unit'])  # Identifier l'échange de manière unique
        if exc_id in explored:
            continue  # Passer cet échange s'il a déjà été exploré

        # Ajouter l'échange à l'ensemble des explorés
        explored.add(exc_id)
        
        # Passer au niveau suivant : récupérer les échanges de l'input de cet échange
        level_input = exc.input
        exchanges_next_level = level_input.exchanges()
        
        # Appeler récursivement la fonction pour traiter les échanges du niveau suivant
        data = process_exchanges_dynamic(exchanges_next_level, data, level + 1, max_level, explored)  # Passer explored ici
    
    return data


# Initialiser l'ensemble des procédés déjà explorés pour tous les scénarios
explored_processes = set()  # <- MODIFICATION

# Créer un DataFrame vide pour chaque scénario
for scenario in scenarios:
    data = pd.DataFrame(columns=['Scenario Process'])  # La première colonne pour les noms des process

    # Filtrer les procédés par scénario et phases
    scenario_processes = []

    for location in locations:
        for phase in phases:
            scenario_processes.append([p for p in case_studies if scenario in p['name'] and phase in p['name'] and location in p['name']][0])

    for process in scenario_processes:
        process_name = process['name']  # Récupérer le nom du process
        
        # Vérifier si le process a déjà été exploré pour éviter les doublons
        if process_name not in explored_processes:  # <- MODIFICATION
            # Ajouter une ligne pour le nom du process
            data = pd.concat([data, pd.DataFrame([[process_name] + [''] * (len(data.columns) - 1)], columns=data.columns)], ignore_index=True)  # <- MODIFICATION
            explored_processes.add(process_name)  # Marquer le process comme exploré
        # Traiter les échanges pour chaque procédé
        data = process_exchanges_dynamic(process.exchanges(), data)  # Notez que 'explored' n'est pas ici

    # Écrire le DataFrame dans un fichier CSV
    data.to_csv(f"recursive_{scenario}_arborescence_procedes.csv", index=False, encoding='utf-8')

# Vider l'ensemble explored après l'écriture des fichiers
explored_processes.clear()  # <- MODIFICATION

print("Fichiers CSV générés avec succès avec le nom des process apparaissant une seule fois.")


In [ ]:
# Récupérer les masses transportées par bateau suivant la phase de distribution

container_ship = [
    activity for activity in eidb
    if 'market for transport, freight, sea, container ship' in activity['name']
    and not 'with' in activity['name']
][0]

offsite_acts = [
    activity for activity in case_studies
    if 'offsite_' in activity['name']
#    and not 'with' in activity['name']
]

scenarios = ['electricity, baseline', 'electricity, alternative', 'heat, baseline', 'heat, alternative']

# Initialiser le dictionnaire des activités
scenario_dict = {}

# Parcourir les scénarios
for scenario in scenarios:
    # Initialiser la liste des activités pour chaque scénario
    scenario_acts = [
        activity for activity in case_studies
        if 'production' in activity['name'] 
        and scenario in activity['name']
        and 'offsite' in activity['name']
    ]
    sc_dist_act = [
        activity for activity in case_studies
        if 'distribution' in activity['name'] 
        and scenario in activity['name']
        and 'offsite' in activity['name']
    ][0]
    # Parcourir chaque activité de la liste scenario_acts
    for scenario_act in scenario_acts:
        # Ajouter l'activité courante dans le dictionnaire
        scenario_dict[scenario_act['name']] = {}
        # Parcourir les échanges de la technosphère de l'activité
        for exc in scenario_act.exchanges():
            # Chercher l'activité pointée par l'échange
            sub_act = [act for act in case_studies if exc.input['name'] in act['name']][0]
            dist_act_name = sub_act['name'].replace("_prod", "_dist")
            multiplier = exc['amount']
            print(sub_act['name'])
            # Calculer la somme des masses positives des échanges de l'activité pointée
            total_mass = 0
            for sub_exc in sub_act.exchanges():
                print(f"    - {sub_exc.input['name']}, {sub_exc['amount']} {sub_exc['unit']}")
                if sub_exc['unit'] == 'kilogram' and sub_exc['amount'] > 0 and sub_exc['type'] != 'biosphere' :
                    total_mass += sub_exc.get('amount', 0)
            
            # Ajouter la somme des masses dans le dictionnaire pour l'activité pointée
            scenario_dict[scenario_act['name']][sub_act['name']] = (total_mass) #* multiplier)                   

# Affichage du dictionnaire final
print(scenario_dict)


In [ ]:
# Afficher les activités multirégionales et les exchanges associés

multi_activities = [act for act in case_studies if 'phase' in act['name']]

# Afficher les activités triées
for activity in multi_activities:
    print(f"Activity: {activity['name']}")
    for exc in activity.technosphere():
        print(f"  - {exc.input['name']}")
#        sub_exc_act = [act for act in case_studies if exc.input['name'] in act['name']][0]# and exc.input['location'] == 'RoW'][0]
#        for sub_exc in sub_exc_act.technosphere():
#            print(f"    - {sub_exc.input['name']}")
    print(' ')

In [ ]:
# Effacer un exchange en particulier

# Trouver l'activité spécifique
activities = [act for act in case_studies if 'electricity production, oil' in act['name']]

# Identifier l'échange à supprimer
for activity in activities:
    for exc in activity.exchanges():
        if 'waste mineral oil' in exc.input['name']:  # Critère pour identifier l'échange
            exc.delete()
            print(f"L'échange '{exc.input['name']}' a été supprimé de l'activité '{activity['name']}'.")
            activity.save()

In [ ]:
# Trouver l'activité spécifique
activities = [act for act in case_studies if 'phase' in act['name']]
activities

In [ ]:
### Récupérer les noms des activités dans un excel en supprimant les préfixes et en évitant les doublons
prefixes = ['offsite_', 'onsite_', 'waste_']
activity_names = [activity['name'] for activity in elec_bl]

# Supprimer les préfixes
cleaned_activity_names = []
for name in activity_names:
    for prefix in prefixes:
        if name.startswith(prefix):
            name = name[len(prefix):]
    cleaned_activity_names.append(name)

# Enlever les doublons
unique_activity_names = list(set(cleaned_activity_names))

# Créer un DataFrame avec les noms sans doublons
df = pd.DataFrame(unique_activity_names, columns=['Activity Name'])

# Écrire le DataFrame dans un fichier Excel
#df.to_excel("activities_names.xlsx", index=False)


In [ ]:
# Trouver une liste d'activités et lister les échanges de chaque activité

# Step 1: Find all activities with 'phase' in their name
phase_activities = [act for act in elec_bl if 'phase' in act['name']]

for activity in phase_activities:
    print(f"Activity: {activity['name']}")
    for exc in activity.exchanges():
        print(f"  - {exc.input['name']}")           


In [ ]:
test = [
    activity for activity in eidb
    if 'market for municipal solid waste' in activity['name']    
    and 'RoW' in activity['location']  
][0]
test

In [ ]:
# Checker les échanges dans une activité
for exc in test.technosphere():
    print(exc.input, exc.amount, exc.output)

In [ ]:
# Supprimer toutes les activités répondant à un critère de la base de données
for activity in elec_bl:
    if ', onsite' in activity['name'] or ', offsite' in activity['name']:
#        activity.delete()
    else:
        pass

In [ ]:
# Trouver une liste d'activités et lister les échanges de chaque activité

# Step 1: Find all activities with 'waste_' in their name
waste_activities = [act for act in elec_bl if 'waste_' in act['name']]

# Step 2: Create a dictionary to map exchanges to the activities where they are found
exchange_to_activities = {}

for activity in waste_activities:
    for exc in activity.exchanges():
        exchange_name = exc.input['name']
        if exchange_name not in exchange_to_activities:
            # Initialize a list for the exchange if it's not already in the dictionary
            exchange_to_activities[exchange_name] = []
        # Add the activity name to the list for this exchange
        exchange_to_activities[exchange_name].append(activity['name'])

# Display the exchanges along with the list of activities in which they are found
for exchange_name, activities in exchange_to_activities.items():
    print(f"Exchange: {exchange_name}, Found in activities: {', '.join(activities)}")

In [ ]:
# Sélectionner une activité dans une bdd
activity_1 = [
    activity for activity in case_studies
    #if 'offsite' in activity['name']
#    if 'electricity, baseline' in activity['name']   
    if 'heat production, wood pellet, at furnace 300kW' in activity['name']
    and not 'onsite' in activity['name']
    and not 'offsite' in activity['name']  

#    and '300' in activity['name']   
#    and activity['location'] == 'RoW'
][0]
activity_1

In [ ]:
for exc in activity_1.exchanges():
    if exc['type']!= "biosphere":
        print(f'{exc.input['name']}, {exc['amount']}, {exc['unit']}, {exc['type']}, {exc.input['location']}')

In [ ]:
# Sélectionner une activité dans une bdd
activity_2 = [
    activity for activity in eidb
    if 'heat production, wood pellet, at furnace 300kW' in activity['name']
#    and 'treatment' in activity['name']   
    and not 'market' in activity['name']   
    and activity['location'] == 'RoW'
][0]
activity_2

In [ ]:
for exc in activity_2.exchanges():
    if exc['type']!= "biosphere":
        print(f'{exc.input['name']}, {exc['amount']}, {exc['unit']}, {exc['type']}, {exc.input['location']}')

In [ ]:
total_amount_test = 0
for exc in activity_6.technosphere():
    if exc['amount']<0:
        print(total_amount_test)
        total_amount_test += exc['amount']
total_amount_test